## Create a client

To get started creating metadata and publishing measurements, the client code must first instantiate a `MetadataStoreClient` and `DataStoreClient`, respectively. The `MetadataStoreClient` is used for creating and querying metadata, and the `DataStoreClient` is used for publishing and querying measurements and conditions.

In [ ]:
# Perform some example-specific setup. This is not needed when writing your own code.
%run ../initialize_example_notebook.py

from ni.datastore.data import DataStoreClient
from ni.datastore.metadata import MetadataStoreClient

metadata_store_client = MetadataStoreClient()
data_store_client = DataStoreClient()

__Recommendation:__ A client can be used within the context of a `with` statement in order to have its lifetime managed for it. Upon exiting the `with` statement, any gRPC channels owned by the client will be closed.

In [ ]:
with MetadataStoreClient() as metadata_store_client_one:
    query_results = metadata_store_client_one.query_hardware_items()  
    # ... Perform any further operations with metadata_store_client_one ...

# metadata_store_client_one will automatically close upon exiting the 'with' block

Alternatively, the `close()` method of the client may be called to perform the same cleanup that occurs when exiting the `with` block above.

In [ ]:
metadata_store_client_two = MetadataStoreClient()

query_results = metadata_store_client_two.query_hardware_items()
# ... Perform any further operations with metadata_store_client_two ...

# Manually close the client when done using it
metadata_store_client_two.close()

Because notebooks such as the current example do not support splitting a single `with` statement across multiple code snippets, this example opts to call `close()` manually in the last code snippet.

## Register a metadata schema

The registration of the metadata schema is usually done by the administrator of the system. The metadata schema is defined in a JSON file, which is then registered in the system. This registration is not usually done as part of running tests or capturing data

In [ ]:
schema_id = metadata_store_client.register_schema_from_file("sample_schema.json")
print(f"registered schema id: {schema_id}")

## Create the test result metadata

Now that a schema has been registered we can create our metadata objects with the expected metadata.  Note, however, the creation of each metadata object must include the required values from the schema.

In [ ]:
import grpc
from ni.datastore.data import TestResult
from ni.datastore.metadata import Operator, TestStation, SoftwareItem

# Invalid creation of operator - no badge_number provided
operator = Operator(name="James Bowery", schema_id=schema_id)
try:
    operator_id = metadata_store_client.create_operator(operator)
except grpc.RpcError as e:
    print("Failed to create operator:")
    print(f"  {e.details()}")

# Add the required attribute to the operator object
operator.extension["badge_number"] = "emp-128256"
operator_id = metadata_store_client.create_operator(operator)

# Invalid creation of test station - location is invalid
test_station = TestStation(name="TestStation_12", schema_id=schema_id)
test_station.extension["location"] = "Texas"
try:
    test_station_id = metadata_store_client.create_test_station(test_station)
except grpc.RpcError as e:
    print("Failed to create test station:")
    print(f"  {e.details()}")

# Fix the location
test_station.extension["location"] = "USA"
test_station_id = metadata_store_client.create_test_station(test_station)

# Invalid creation of software item -  software license is invalid (not matching the pattern defined in the schema)
software_item = SoftwareItem(product="Windows", version="10.0.19044", schema_id=schema_id)
software_item.extension["license"] = "enterprise_LIC"
try:
    software_item_id = metadata_store_client.create_software_item(software_item)
except grpc.RpcError as e:
    print("Failed to create software item:")
    print(f"  {e.details()}")

# Fix the software license to create the software item
software_item.extension["license"] = "LIC_enterprise"
software_item_id = metadata_store_client.create_software_item(software_item)
    
test_result = TestResult(
    name="sample publish test result",
    operator_id=operator_id,
    test_station_id=test_station_id,
    schema_id=schema_id,
    software_item_ids=[software_item_id])
test_result.extension["session_file_path"] = "C:\\my_test_description.xlsx"


## Create the test result object

Now that we have the test result metadata, we can create the test result object. Note below that schema validation can fail if your metadata does not match what's specified in the schema.

In [ ]:
test_result_id = data_store_client.create_test_result(test_result)
print(f"created test result id: {test_result_id}")

## Publish the data

Now that we have a valid test result and schema, we can publish our data

In [ ]:
from datetime import timezone
import hightime as ht
from nitypes.waveform import AnalogWaveform
from nitypes.waveform import Timing
import numpy as np
from ni.datastore.data import Step

name = "data publish sample"
waveform = AnalogWaveform(
    sample_count=3,
    raw_data=np.array([1.0, 2.0, 3.0]),
    timing=Timing.create_with_regular_interval(
        ht.timedelta(seconds=1e-3),
        ht.datetime.now(timezone.utc)
    )
)
step = Step(name="Initial step", test_result_id=test_result_id)
step_id = data_store_client.create_step(step)
published_measurement_id = data_store_client.publish_measurement(
    name=name,
    value=waveform,
    step_id=step_id,
)
print(f"The published measurement id is {published_measurement_id}")

## Query the data

Now that we've got data in the database, we can query for it using our OData query api.

In [ ]:
from nitypes.waveform import AnalogWaveform

published_measurements = data_store_client.query_measurements(odata_query=f"$filter=id eq {published_measurement_id}")
found_measurement = next(iter(published_measurements), None)

if found_measurement is not None:
    test_result = data_store_client.get_test_result(found_measurement.test_result_id)
    operator = metadata_store_client.get_operator(test_result.operator_id)
    # badge_number is a custom attribute in the schema
    # It's not a standard attribute of the operator object
    badge_number = operator.extension["badge_number"]
    print(f"operator {operator.name}'s badge number is: {badge_number}")

    waveform = data_store_client.read_data(found_measurement, expected_type=AnalogWaveform)
    print(f"published data is: {waveform.raw_data}")

## Close the clients

In [ ]:
metadata_store_client.close()
data_store_client.close()

# Perform some example-specific cleanup. This is not needed when writing your own code.
data_store_context.close()